In [2]:
#Pre-Processing

import os
import argparse
import networkx as nx
import math        
import numpy as np

def state_to_numpy(state):
    strlist = state.split()
    val_list = [float(s) for s in strlist]
    return np.array(val_list)

def list_all_dir(data_dir):
    task_dirs = os.listdir(data_dir)

    list_dir = []
    for task_dir in task_dirs:
        env_dirs = os.listdir(data_dir+"/"+task_dir)
        for env_dir in env_dirs:
            list_dir.append(data_dir +"/"+ task_dir +"/"+ env_dir)
    return list_dir  

def process_it(G, directory):
    start = np.loadtxt(directory+"/start_nodes.txt")
    goal = np.loadtxt(directory+"/goal_nodes.txt")
    occ_grid = np.loadtxt(directory+"/occ_grid.txt")
    # occ_grid = occ_grid.split(",")
    path_nodes = []
    i = 0
    all_data = []
    with open(directory + "/path_nodes.txt", 'r') as file:
        lines  = file.readlines()
        for line in lines:
            line = line.strip('\n')
#             print(line)
#             print("\n\n")
            
            s = state_to_numpy(G.node[str(int(start[i]))]['state'])
            g = state_to_numpy(G.node[str(int(goal[i]))]['state'])
            og = occ_grid[i]
            path_nodes = str(line).split(",")
            # print(path_nodes)
            for path_node in path_nodes:
                if(path_node=='-1'):
                    continue
                node_conf = state_to_numpy(G.node[path_node]['state'])
                curr_node = np.array([])
                # print("Data = ",node_conf, s, g, occ_grid)
#                     print("\n")
#                     print("node_conf = ", node_conf, " s = ", s, " g = ",g)

                curr_node = np.concatenate((node_conf, s, g, og))
#                     print("shape of curr_node = ", curr_node.shape)
                all_data.append(curr_node)
            i+=1
    return all_data

In [4]:
# Workspace problem with several narrow gaps

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import os
import csv
from random import randint, random, seed
import time

# (restrict tensorflow memory growth)
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

# neural network parameters
mb_size = 256
h_Q_dim = 512
h_P_dim = 512

c = 0
# learning rate
lr = 1e-4

# problem dimensions
dim = 2
dataElements = dim*3 + 100 # sample (2D), init(2D), goal(2D), occup_grid(400) 

z_dim = 2 # latent
X_dim = dim # samples
y_dim = dim # reconstruction of the original point
c_dim = dataElements - dim # dimension of conditioning variable

In [6]:
G = nx.read_graphml("graphs/dense_graph.graphml")
data_dir = "dataset_23June"

directory = data_dir
print(directory)
final_data = []
flag = 0

data = np.array(process_it(G, directory))

# np.random.shuffle(data)
print("shape of array: ",data.shape)

numEntries = data.shape[0]

dataset_23June
('shape of array: ', (44722, 106))


In [8]:
# split the inputs and conditions into test train (to be processed in the next step into an occupancy grid representation)
ratioTestTrain = 0.8;
numTrain = int(numEntries*ratioTestTrain)

X_train = data[0:numTrain,0:dim] # state: x, y, z, xdot, ydot, zdot
c_train = data[0:numTrain,dim:dataElements] # conditions: gaps, init (6), goal (6)
# print("c_train = ",c_train[:,:4])

X_test = data[numTrain:numEntries,0:dim]
c_test = data[numTrain:numEntries,dim:dataElements]

#########################################################
c_train1 = []
c_test1 = []
c_train1 = c_train
c_test1 = c_test
#########################################################
numTest = X_test.shape[0]
# print(data[:,:6])
# print(c_test[:,:4])

# print("shape of final obstacle = ",obs.shape)
print("shape of c_train1 = ", c_train1.shape)
print("shape of c_test1 = ",c_test1.shape)

('shape of c_train1 = ', (35777, 104))
('shape of c_test1 = ', (8945, 104))


In [10]:
# define networks
print("X_dim = ",X_dim)
print("c_dim = ",c_dim)
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, X_dim])
c = tf.placeholder(tf.float32, shape=[None, c_dim])
    
# Q
inputs_Q = tf.concat(axis=1, values=[X,c])

dense_Q1 = tf.layers.dense(inputs=inputs_Q, units=h_Q_dim, activation=tf.nn.relu)
dropout_Q1 = tf.layers.dropout(inputs=dense_Q1, rate=0.5)
dense_Q2 = tf.layers.dense(inputs=dropout_Q1, units=h_Q_dim, activation=tf.nn.relu)

z_mu = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_mu
z_logvar = tf.layers.dense(inputs=dense_Q2, units=z_dim) # output here is z_logvar

# P
eps = tf.random_normal(shape=tf.shape(z_mu))
z = z_mu + tf.exp(z_logvar / 2) * eps
inputs_P = tf.concat(axis=1, values=[z,c])

dense_P1 = tf.layers.dense(inputs=inputs_P, units=h_P_dim, activation=tf.nn.relu)
dropout_P1 = tf.layers.dropout(inputs=dense_P1, rate=0.5)
dense_P2 = tf.layers.dense(inputs=dropout_P1, units=h_P_dim, activation=tf.nn.relu)

y = tf.layers.dense(inputs=dense_P2, units=X_dim) # fix to also output y

# training
########### comment in the one with 0 weight and uncomment the other ###########
w = [[1, 1]];
# w = [[1, 1, 1, 0, 0, 0]];
recon_loss = tf.losses.mean_squared_error(labels=X, predictions=y, weights=w)

# TODO: fix loss function for angles going around
kl_loss = 10**-4 * 2 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)

cvae_loss = tf.reduce_mean(kl_loss + recon_loss)

train_step = tf.train.AdamOptimizer(lr).minimize(cvae_loss)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
it = 0;

('X_dim = ', 2)
('c_dim = ', 104)


In [11]:
saver = tf.train.Saver()
path_ = os.getcwd() + "/checkpoints_NS/model.ckpt"
print("path = ",path_)
print("numTrain = ",numTrain)
try:
    saver.restore(sess, path_)
    print("Model Restored!!")
except Exception as e:
    print("Could not restore checkpoint!")
    print(e)
x1 = []
y1 = []    
print("z_dim = ", z_dim)
print("c_dim = ", c_dim)
print("c_train = ", c_train.shape)
for it in range(it,it+600001):
#     print("c_dim = ",c_dim)
    # randomly generate batches
    batch_elements = [randint(0,numTrain-1) for n in range(0,mb_size)]
    X_mb = X_train[batch_elements,:]
    c_mb = c_train1[batch_elements,:]

    _, loss, r = sess.run([train_step, cvae_loss, recon_loss], feed_dict={X: X_mb, c: c_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('Loss: {:.4}'. format(loss))
        x1.append(it)
        y1.append(loss)
        print("recon_loss = ", r)
    if it % 1000 == 0:    
        saver.save(sess, path_)
        print("saved session to ", path_)

('path = ', '/home/vernwalrahul/projects/Workspace_RF_2d/checkpoints_NS/model.ckpt')
('numTrain = ', 35777)
INFO:tensorflow:Restoring parameters from /home/vernwalrahul/projects/Workspace_RF_2d/checkpoints_NS/model.ckpt
Model Restored!!
('z_dim = ', 2)
('c_dim = ', 104)
('c_train = ', (35777, 104))
Iter: 0
Loss: 0.001302
('recon_loss = ', 0.00032113987)
('saved session to ', '/home/vernwalrahul/projects/Workspace_RF_2d/checkpoints_NS/model.ckpt')


KeyboardInterrupt: 

In [14]:
#to check if a node is free
def is_free(node_pos, obstacles):
    flag = 1
    eps = 0.04
    for obs in obstacles:
        x1, y1, x2, y2 = obs
        if(node_pos[0] < x2 + eps and node_pos[0] > x1 - eps):
            if(node_pos[1] < y2 + eps and node_pos[1] > y1 - eps):
                flag = 0
                return flag
    return flag

################Create Occ_Grid
def get_occ_grid(obstacles):
    occ_grid = np.ones((10,10), dtype=int)
    eps = 0.05
    for i in range(0,10):
        for j in range(0, 10):
            if(not (is_free((i/10.0+eps,j/10.0+eps), obstacles))):
                occ_grid[i,j] = 0
            else:
                occ_grid[i,j] = 1
    return occ_grid.ravel()

obs1 = [0.0, 0.1, 0.4, 0.2]
obs2 = [0.5, 0.1, 0.6, 0.2]
obs3 = [0.7, 0.1, 1.0, 0.2]

init_ = np.array([0.05 , 0.95])
goal_ = np.array([0.95 , 0.95])

c_here = []
for x in range(1,7):
    obs4 = [x/10.0, 0.5, x/10.0+0.1, 1.0]
    obs5 = [x/10.0, 0.2, x/10.0+0.1, 0.4]
    obs6 = [x/10.0, 0.0, x/10.0+0.1, 0.1]
    
    obstacles = [obs1, obs2, obs3, obs4, obs5, obs6]
    curr_occ_grid = np.array(get_occ_grid(obstacles))
    print(curr_occ_grid.reshape(10,10))
    c_here.append(np.concatenate((init_, goal_, curr_occ_grid)))

[[1 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]]
[[1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]]
[[1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 1 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]]
[[1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [0 1 0 0 1 0 0 0 0 0]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]]
[[1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 0 1

In [41]:
# plot the latent space
num_viz = 100
# seed(1000)
print("c_test.shape = ",c_test.shape)
print("c_train.shape = ",c_train.shape)
# print(c_test[:,:4])
vizIdx = randint(0,numTest-1);
# vizIdx = 2180
print vizIdx
c_sample_seed = data[numTrain+vizIdx,2:]
print(c_sample_seed)
print("c_sample_seed = ", c_sample_seed[:4])
occ_g = c_sample_seed[4:].reshape(10,10)

# c_here = np.array(c_here)
# c_sample_seed = c_here[0,:]
# occ_g = c_here[0,4:].reshape(10,10)

ts = X_test[vizIdx,:2]
print("Train Sample = ", ts)

# c_sample_seed = c_test[vizIdx,:]
# occ_g = c_test[vizIdx,4:].reshape(20,20)
# print("Testing Sample = ",X_test[vizIdx,:2])

# print(c_sample_seed[:4])
init = c_sample_seed[:2]
goal = c_sample_seed[2:4]
c_sample = np.repeat([c_sample_seed],num_viz,axis=0)

print("c_sample.shape = ",c_sample.shape)
# directly sample from the latent space (preferred, what we will use in the end)
y_viz, z_viz = sess.run([y, z], feed_dict={z: np.random.randn(num_viz, z_dim), c: c_sample})

fig1 = plt.figure(figsize=(10,6), dpi=80)
ax1 = fig1.add_subplot(111, aspect='equal')

plt.scatter(y_viz[:,0],y_viz[:,1], color="green", s=20)
plt.title(str(numTrain+vizIdx)+","+str(vizIdx))

for i in range(10):
        for j in range(10):
            if(occ_g[i,j]==0):
                ax1.add_patch(patches.Rectangle(
                (i/10.0, j/10.0),   # (x,y)
                0.1,          # width
                0.1,          # height
                alpha=0.6
                ))

plt.scatter(init[0], init[1], color="red", s=100, edgecolors='black') # init
plt.scatter(goal[0], goal[1], color="blue", s=100, edgecolors='black') # goal
plt.scatter(ts[0], ts[1], color="orange", s=50, edgecolors='black') # goal

plt.xlim(0,1)
plt.ylim(0,1)

plt.savefig("testing_"+str(numTrain+vizIdx)+".jpg", bbox_inches='tight')
plt.show()

('c_test.shape = ', (8945, 104))
('c_train.shape = ', (35777, 104))
4267
[0.01262756 0.6964687  0.02483459 0.16606175 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         0.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.
 1.         1.         1.         1.         0.         0.
 0.         0.         0.         0.         1.         0.
 0.         0.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.
 1.         1.         1.         1.      

In [12]:
# conditions = []
# t_cases = [33396, 37596, 37206, 37214, 35591, 32447, 33265, 34226, 31937, 38062, 37070, 34959, 34088, 36841, 31974, 35897, 38648, 32513, 38753, 37170]
# print(data[3396,2:])
# for t in t_cases:
#     conditions.append(data[t,2:])
# # print(conditions)
# print(len(conditions))
# print(conditions[0])
# conditions = np.array(conditions)
# print(conditions.shape)
# np.savetxt("test_cases.txt", conditions, delimiter = " ", fmt = "%s")

def load_conditions(directory):
    start_posns = np.loadtxt(directory+"/start_posns.txt", delimiter=" ")
    goal_posns = np.loadtxt(directory+"/goal_posns.txt", delimiter=" ")
    occ_grid = np.loadtxt(directory+"/occ_grid.txt", delimiter=" ")
    
    cond = np.concatenate((start_posns, goal_posns, occ_grid), axis = 1)
    
    print("cond.shape = ", cond.shape)
    return cond

conditions = load_conditions("test_dataset_23June/easy")

('cond.shape = ', (100, 104))


In [13]:
# conditions = np.loadtxt("test_cases.txt", delimiter = " ")
print(conditions)

[[0.02441759 0.93227395 0.75757127 ... 1.         1.         1.        ]
 [0.26861565 0.70930449 0.7733626  ... 1.         1.         1.        ]
 [0.18471032 0.8201976  0.96368896 ... 1.         1.         1.        ]
 ...
 [0.94582876 0.81263927 0.10186312 ... 1.         1.         1.        ]
 [0.17707145 0.83243238 0.89689576 ... 1.         1.         1.        ]
 [0.20330311 0.45177786 0.35919237 ... 1.         1.         1.        ]]


In [17]:
def state_to_numpy(state):
    strlist = state.split()
#     print("strlist = ", strlist)
    val_list = [float(s) for s in strlist]
    return np.array(val_list)

def edge_to_configs(state1, state2):
    EDGE_DISCRETIZATION = 20
    config1 = state_to_numpy(state1)
    config2 = state_to_numpy(state2)

    diff = config2 - config1
    step = diff/EDGE_DISCRETIZATION

    to_check = list()
    to_check.append(config1)

    for i in xrange(EDGE_DISCRETIZATION - 1):
        conf = config1 + step*(i+1)
        to_check.append(conf)

    return to_check

#get output node posns
def get_o_node_posns(c_sample_seed, num_viz, count):
    return []
    init = c_sample_seed[:2]
    goal = c_sample_seed[2:4]
    occ_grid = c_sample_seed[4:].reshape(10,10)
    c_sample = np.repeat([c_sample_seed],num_viz,axis=0)
    y_viz, z_viz = sess.run([y, z], feed_dict={z: np.random.randn(num_viz, z_dim), c: c_sample})
    
    fig1 = plt.figure(figsize=(10,6), dpi=80)
    ax1 = fig1.add_subplot(111, aspect='equal')
    
    free_nodes = []
    y_viz = np.array(y_viz)
    y_viz = list(y_viz)
#     print("y_viz = ",y_viz)
    for cc in y_viz:
            eps = 0.015
            x_, y_ = float(cc[0]), float(cc[1])
            
            x_ = min(0.95, cc[0])
            y_ = min(0.95, cc[1])
            if(occ_grid[int((x_+eps)*10), int((y_+eps)*10)]==0 or occ_grid[int((x_-eps)*10), int((y_-eps)*10)]==0):
                continue
            else:
                free_nodes.append(cc)
    y_viz = np.array(free_nodes)
    
    plt.scatter(y_viz[:,0],y_viz[:,1], color="green", s=20)
    
    for i in range(10):
            for j in range(10):
                if(occ_grid[i,j]==0):
                    ax1.add_patch(patches.Rectangle(
                    (i/10.0, j/10.0),   # (x,y)
                    0.1,          # width
                    0.1,          # height
                    alpha=0.6
                    ))
    
    plt.scatter(init[0], init[1], color="red", s=100, edgecolors='black') # init
    plt.scatter(goal[0], goal[1], color="blue", s=100, edgecolors='black') # goal
    
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.title(str(count))
#     plt.savefig("test_dataset_23June-1/medium/images/RF_numviz_"+str(num_viz)+"_"+str(count)+".jpg", bbox_inches='tight')
#     plt.show()
    
    return y_viz

#load shallow_graph
def load_halton_samples(num_viz):
    shallow_G = nx.read_graphml("graphs/halton2D"+str(num_viz)+"_1.graphml")
    shallow_G.remove_edges_from(list(shallow_G.edges()))
    return shallow_G

def remove_invalid_edges(G, occ_grid):
    
    # print("total no of edges = ", len(list(G.edges())))
    to_remove = []
    for edge in G.edges():
        u, v = edge
        state1 = G.node[u]['state']
        state2 = G.node[v]['state']
        configs_to_check = edge_to_configs(state1,state2)

        edge_free = 1
        eps = 0.015
        for cc in configs_to_check:
            cc[0] = min(0.95, cc[0])
            cc[1] = min(0.95, cc[1])
            if(occ_grid[int((cc[0]+eps)*10), int((cc[1]+eps)*10)]==0 or occ_grid[int((cc[0]-eps)*10), int((cc[1]-eps)*10)]==0):
                edge_free = 0
                break
        if(not edge_free):
            to_remove.append((u, v))

    for r in to_remove:
        G.remove_edge(r[0], r[1])
    
#     print("removed ", len(to_remove), "edges")
        
    return G

In [96]:
import time
def calc_weight(s, g):
    return sqrt(np.sum((s-g)**2))

def connect_knn(G, K):
#     print("no of nodes = ", len(list(G.nodes())))
    G1 = G.copy()
    nodes = []
    states = []
    
#     t1 = time.time()
    for node in G.nodes():
        state = G.node[node]['state']
        conf = state_to_numpy(state)

        for k in range(K):
            w = 1000000
            sn = None
            for node1 in G1.nodes():
            	if(node == node1):
            		continue
                state1 = G1.node[node1]['state']
#                 print("node1 = ",node1)
#                 print("state1 = ",state1)
                
                conf1  = state_to_numpy(state1)
                if(calc_weight(conf, conf1) < w):
                    w = calc_weight(conf, conf1)
                    sn = node1
            # if(check_for_collision(node, sn)==1):
            G.add_edge(node, sn)
            # print("connected edge from ",node, " to ",sn)
            G[node][sn]['weight'] = w
            
            nodes.append(sn)
            states.append(G1.node[sn]['state'])
            G1.remove_node(sn)
    
        for i in range(len(nodes)):
            G1.add_node(nodes[i], state = states[i])
        
        nodes = []
        states = []
        
#     t2 = time.time()
#     print("no of nodes = ", len(list(G.nodes())), " no of edges = ", len(list(G.edges)))
#     print("connected knn in time t = ", t2-t1)
    return G

def get_path_length(shallow_G, o_node_posns, src_posn, goal_posn, occ_grid, k):
    G = shallow_G.copy()
    sx, sy = src_posn[0], src_posn[1]
    gx, gy = goal_posn[0], goal_posn[1]
    G.add_node('s', state = str(sx)+" "+str(sy))
    G.add_node('g', state = str(gx)+" "+str(gy))
    for i in range(len(o_node_posns)):
        x,y = o_node_posns[i][0], o_node_posns[i][1]
        G.add_node('o'+str(i), state = str(x)+" "+str(y))
#     print("connecting knn")    
    G = connect_knn(G, k)
#     print("connected knn")
    print("no of edges = ", len(list(G.edges())))
    t1
    G = remove_invalid_edges(G, occ_grid)
    path_length = None
    path_node_posns = []
    print("total no of nodes = ", len(list(G.nodes())))
    try:
        path_length = nx.dijkstra_path_length(G,'s','g', weight = 'weight')
        path_nodes = nx.dijkstra_path(G, 's', 'g', weight = 'weight')
        for node in path_nodes:
            path_node_posns.append(state_to_numpy(G.node[node]['state']))
    except Exception as e:
        pass
    return path_length, path_node_posns

In [97]:
from random import randint
from math import sqrt
import time
# cond_test = []
# test_cases = []
# for x in range(10):
#     test_cases.append(randint(0,numTest-1))

# test_cases = [1379]

num_v = [600]
K = [5, 7, 10]

for num_viz in num_v:
    print("--------------------------------Num_Viz = ", num_viz)
    for k in K:
        print("--------------------------------------k = ",k)
        shallow_G = load_halton_samples(num_viz)
        path_lengths = []
        all_path_nodes_posns = []
        count = 0
        # print(test_cases)
        # print(conditions[0])
        # return
        failed_c = 0
        path_found = []
        path_lengths = []
        for cond in conditions:
        #     print("---------------------------------------cond = ",cond)
        #     if(count<7):
        #         count+=1
        #         continue
#             print("count = ", count)
            o_node_posns = get_o_node_posns(cond, num_viz, count)
#             print("got node posns")
            occ_grid = cond[4:].reshape(10,10)
        #     print(occ_grid)
        #     break
        #     print(o_node_posns)
        
            path_length, path_node_posns = get_path_length(shallow_G, o_node_posns, cond[:2], cond[2:4], occ_grid, k)
        #     path_lengths.append(path_length)
            if(path_length==None):
                failed_c += 1
                path_found.append(0)
                path_lengths.append(-1)
            else:
                path_found.append(1)
                path_lengths.append(path_length)
            print("path_length = ", path_length)
            all_path_nodes_posns.append(path_node_posns)
            count += 1
        print("num_viz = ", num_viz, "k = ", k)
        print("failed = ", failed_c)
        print("path_lengths = ", path_lengths)
        np.savetxt("test_dataset_23June/easy/path_lengths_halton_numviz"+str(num_viz)+"_k"+str(k)+".txt", path_lengths)

('--------------------------------Num_Viz = ', 600)
('--------------------------------------k = ', 5)
('no of edges = ', 1707)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1707)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1704)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1708)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1706)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1705)
('total no of nodes = ', 602)
('path_length = ', 0.3665151949648047)
('no of edges = ', 1703)
('total no of nodes = ', 602)
('path_length = ', 0.9910974262127399)
('no of edges = ', 1709)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1705)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1706)
('total no of nodes = ', 602)
('path_length = ', 0.5668889679323099)
('no of edges = ', 1705)
('total no of nodes = ', 602)
(

('no of edges = ', 1704)
('total no of nodes = ', 602)
('path_length = ', 0.6653598844761854)
('no of edges = ', 1705)
('total no of nodes = ', 602)
('path_length = ', 0.44518935211175703)
('no of edges = ', 1703)
('total no of nodes = ', 602)
('path_length = ', 0.9912683570746736)
('no of edges = ', 1704)
('total no of nodes = ', 602)
('path_length = ', 0.625246787891016)
('no of edges = ', 1704)
('total no of nodes = ', 602)
('path_length = ', 0.4299380864049779)
('no of edges = ', 1703)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1708)
('total no of nodes = ', 602)
('path_length = ', 0.9178063460067779)
('no of edges = ', 1706)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1708)
('total no of nodes = ', 602)
('path_length = ', None)
('no of edges = ', 1705)
('total no of nodes = ', 602)
('path_length = ', 0.43925246258663386)
('num_viz = ', 600, 'k = ', 5)
('failed = ', 25)
('path_lengths = ', [-1, -1, -1, -1, -1, 0.366515194

('no of edges = ', 2374)
('total no of nodes = ', 602)
('path_length = ', 0.8165386054508214)
('no of edges = ', 2382)
('total no of nodes = ', 602)
('path_length = ', 0.6252197788503255)
('no of edges = ', 2376)
('total no of nodes = ', 602)
('path_length = ', 0.6152763890891806)
('no of edges = ', 2378)
('total no of nodes = ', 602)
('path_length = ', 0.6580797043741289)
('no of edges = ', 2375)
('total no of nodes = ', 602)
('path_length = ', 0.6064532347705964)
('no of edges = ', 2378)
('total no of nodes = ', 602)
('path_length = ', 1.292415420549356)
('no of edges = ', 2378)
('total no of nodes = ', 602)
('path_length = ', 1.0405516148816742)
('no of edges = ', 2376)
('total no of nodes = ', 602)
('path_length = ', 1.4483406449765777)
('no of edges = ', 2375)
('total no of nodes = ', 602)
('path_length = ', 1.0870701338688702)
('no of edges = ', 2375)
('total no of nodes = ', 602)
('path_length = ', 0.7272709485141012)
('no of edges = ', 2378)
('total no of nodes = ', 602)
('path

('no of edges = ', 3330)
('total no of nodes = ', 602)
('path_length = ', 1.0198102305367)
('no of edges = ', 3331)
('total no of nodes = ', 602)
('path_length = ', 0.9497395331752282)
('no of edges = ', 3324)
('total no of nodes = ', 602)
('path_length = ', 1.2640162379415567)
('no of edges = ', 3326)
('total no of nodes = ', 602)
('path_length = ', 0.6795455418290962)
('no of edges = ', 3326)
('total no of nodes = ', 602)
('path_length = ', 0.3031858023620377)
('no of edges = ', 3326)
('total no of nodes = ', 602)
('path_length = ', 0.9583476553878648)
('no of edges = ', 3327)
('total no of nodes = ', 602)
('path_length = ', 0.48979251778149496)
('no of edges = ', 3323)
('total no of nodes = ', 602)
('path_length = ', 0.48815829648975556)
('no of edges = ', 3331)
('total no of nodes = ', 602)
('path_length = ', 0.5003987526147641)
('no of edges = ', 3331)
('total no of nodes = ', 602)
('path_length = ', 0.7419998525001554)
('no of edges = ', 3324)
('total no of nodes = ', 602)
('path

In [46]:
print("failed = ", failed_c)
print("path_lengths = ", path_lengths)
np.savetxt("test_dataset_23June/medium/path_lengths_RF_numviz"+str(num_viz)+"_k"+str(k)+".txt", path_lengths)

('failed = ', 6)
('path_lengths = ', [1.227447567407999, 1.284003217810725, 1.0954510484702686, 0.9683409451473315, 0.9926545729653716, 1.5011914703361025, 1.136285413558691, 0.9029150865622302, 1.3183770325597635, 1.2449785079873008, 1.929550557589348, 1.8325532288067596, 1.2018123754498231, 1.5469782381063708, 1.7036434624356436, 1.0817567399359391, 1.161322027494859, 1.428290131190669, 1.4452209296324043, 1.1868221380708726, 1.083411395873847, 1.4189876856254906, 0.9832885541468575, 1.1273404863753014, 1.0462099575218087, 1.2105568314319834, 1.1785472552239007, 1.3013620728859505, 1.017320080109644, 1.6313805373710206, 0.9995909847657488, 1.125698962237227, 1.2504333856446053, 0.7892738656085072, 1.1561019455794348, -1, 0.7958307835886808, 1.0397893136836023, 0.7874786616184697, 1.4463817798093384, 1.4651742790339275, 0.9916174586698538, 0.8247127295396972, 1.039028841386735, 1.0338337442268302, 1.1223283134921302, 1.1259577019891593, 1.4200927876780611, -1, 1.3713178550897598, 1.25

In [53]:
print(test_cases)
print(numTrain+4396)
print(c_test[4396,:4])
print(data[numTrain+4396,2:6])

[351, 300, 3355, 1546, 281, 2124, 2449, 36, 5276, 2299, 4396]
36274
[0.06047912 0.41480432 0.94036193 0.26299819]
[0.06047912 0.41480432 0.94036193 0.26299819]


In [71]:
for i in range(len(test_cases)):
    cond = c_test[test_cases[i],:]
    occ_grid = cond[4:].reshape(10,10)
    node_posns = np.array(all_path_nodes_posns[i])
    init = cond[:2]
    goal = cond[2:4]
    
    fig1 = plt.figure(figsize=(10,6), dpi=80)
    ax1 = fig1.add_subplot(111, aspect='equal')
    
    plt.scatter(node_posns[:,0], node_posns[:,1], color="green", s=20)
    
    for i in range(10):
            for j in range(10):
                if(occ_g[i,j]==0):
                    ax1.add_patch(patches.Rectangle(
                    (i/10.0, j/10.0),   # (x,y)
                    0.1,          # width
                    0.1,          # height
                    alpha=0.6
                    ))
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.scatter(init[0], init[1], color="red", s=100, edgecolors='black') # init
    plt.scatter(goal[0], goal[1], color="blue", s=100, edgecolors='black') # goal
    plt.show()